<a href="https://colab.research.google.com/github/ZohebAbai/DeepLearning-Projects/blob/master/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Pytorch built-ins functions

In [0]:
import numpy as np
import torch
import torch.nn as nn

`torch.nn` contains utility classes for building neural networks.

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,67,43], [91,88,64], [87,134,58],
                  [102,43,37],[69,96,70],[73,67,43],
                  [91,88,64], [87,134,58],[102,43,37],
                  [69,96,70], [73,67,43], [91,88,64],
                  [87,134,58],[102,43,37], [69,96,70]],
                 dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56,70], [81,101], [119,133],
                   [22,37],[103,119],[56,70],
                   [81,101],[119,133],[22,37],
                   [103,119],[56,70],[81,101],
                   [119,133],[22,37],[103,119]],
                  dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Load the Data

We’ll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [0]:
from torch.utils.data import TensorDataset

In [4]:
#Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

We’ll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [0]:
from torch.utils.data import DataLoader

In [0]:
#Define data loader
batch_size= 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

### Build model

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [7]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4264, -0.3537,  0.4066],
        [-0.1913, -0.0152,  0.4384]], requires_grad=True)
Parameter containing:
tensor([0.0344, 0.2587], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [8]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4264, -0.3537,  0.4066],
         [-0.1913, -0.0152,  0.4384]], requires_grad=True),
 Parameter containing:
 tensor([0.0344, 0.2587], requires_grad=True)]

### Train the model

We are now ready to train the model. We’ll follow the exact same process to implement gradient descent:

* Generate predictions
* Calculate the loss
* Compute gradients w.r.t the weights and biases
* Adjust the weights by subtracting a small quantity proportional to the gradient
* Reset the gradients to zero

The only change is that we’ll work batches of data, instead of processing the entire training data in every iteration. Let’s define a utility function fit which trains the model for a given number of epochs.

In [0]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
  
  #Repeat for given number of epochs
  for epoch in range(num_epochs):
    
    #Train with batches of data
    for xb,yb in train_dl:
      
      #1. Generate predictions
      pred = model(xb)
      
      #2. Calculate loss
      loss = loss_fn(pred, yb)
      
      #3. Compute gradients
      loss.backward()
      
      #4. Updatte parameters using gradients
      opt.step()
      
      #5. Reset the gradients to zero
      opt.zero_grad()
      
    # Print the progress
    if (epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    

In [0]:
# Loss function
import torch.nn.functional as F
loss_fn = F.mse_loss

In [0]:
# Optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [12]:
# Train for 100 epochs
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 179.0728
Epoch [20/100], Loss: 233.3368
Epoch [30/100], Loss: 241.2428
Epoch [40/100], Loss: 70.0048
Epoch [50/100], Loss: 42.6064
Epoch [60/100], Loss: 37.9880
Epoch [70/100], Loss: 24.7659
Epoch [80/100], Loss: 17.5131
Epoch [90/100], Loss: 17.8535
Epoch [100/100], Loss: 14.6051


### Compare predictions with targets

In [13]:
preds = model(inputs)
preds

tensor([[ 58.0315,  71.1795],
        [ 83.9225, 100.3657],
        [113.8391, 132.1493],
        [ 25.8450,  41.8132],
        [102.1006, 115.8672],
        [ 58.0315,  71.1795],
        [ 83.9225, 100.3657],
        [113.8391, 132.1493],
        [ 25.8450,  41.8132],
        [102.1006, 115.8672],
        [ 58.0315,  71.1795],
        [ 83.9225, 100.3657],
        [113.8391, 132.1493],
        [ 25.8450,  41.8132],
        [102.1006, 115.8672]], grad_fn=<AddmmBackward>)

In [14]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.